In [ ]:
import xarray as xr


In [3]:
from anemoi.datasets import open_dataset

In [4]:
file = f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2013/norkyst800-20130101.nc'
ds_fasit = xr.open_dataset(file).isel(s_rho = -1, time = 0 )
ds_fasit.Y

<xarray.DataArray 'Y' (Y: 1148)> Size: 9kB
array([0.000e+00, 8.000e+02, 1.600e+03, ..., 9.160e+05, 9.168e+05, 9.176e+05],
      shape=(1148,))
Coordinates:
    time     datetime64[ns] 8B 2013-01-01
    s_rho    float64 8B -0.004904
  * Y        (Y) float64 9kB 0.0 800.0 1.6e+03 ... 9.16e+05 9.168e+05 9.176e+05
Attributes:
    axis:           Y
    standard_name:  projection_y_coordinate
    units:          meter

In [5]:
#zeta for different points, first I try X = 360 and Y = 760
ds_fasit.zeta.values

array([[  nan,   nan,   nan, ..., -0.87, -0.87, -0.87],
       [  nan,   nan,   nan, ..., -0.87, -0.87, -0.87],
       [  nan,   nan,   nan, ..., -0.87, -0.87, -0.87],
       ...,
       [  nan,   nan,   nan, ...,  0.02,  0.02,  0.02],
       [  nan,   nan,   nan, ...,  0.02,  0.02,  0.02],
       [  nan,   nan,   nan, ...,  0.02,  0.02,  0.02]],
      shape=(1148, 2747), dtype=float32)

In [125]:
file_zarr = f"/lustre/storeB/project/fou/hi/foccus/datasets/norkystv3_hindcast_2013_surface.zarr"
ds_zarr = open_dataset(file_zarr, select = "zeta", start = '2013-01-01T00:00:00', end = '2013-01-01T00:00:00')

In [105]:
import numpy as np

In [ ]:
ds_zarr.statistics

{'mean': array([-0.12241766]),
 'stdev': array([0.46485961]),
 'maximum': array([2.73000002]),
 'minimum': array([-3.30999994])}

In [ ]:
#Trying to convert the dataarray from xr to np to flatten it so it can be used together with the zarr files. 
import matplotlib.pyplot as plt
ds_fasit_1d = ds_fasit["zeta"].values.flatten()
ds_fasit_1d.shape


(3153556,)

In [ ]:
ds_zarr.shape

(2, 1, 1, 3153556)

In [ ]:
ds_fasit.dims

FrozenMappingWarningOnValuesAccess({'Y': 1148, 'X': 2747, 's_w': 41})

In [111]:
ds_zarr.shape

(1, 1, 1, 3153556)

In [ ]:
#the shapes are now similiar
#I am therefore calculating the difference between the datasets and checking wether they are the same or not
diff = ds_fasit_1d - ds_zarr
replace_nan = np.where(np.isnan(diff), 0, diff)
if np.any(replace_nan != 0):
    print("not the same zeta values between the norkyst and zarr model")
else:
    print("the values are the same for the zarr and the norkyst model")

the values are the same for the zarr and the norkyst model


In [138]:
#np.set_printoptions(threshold=np.inf)
#double check manually that values actually are the same

In [129]:
#checking for an another time as well
ds_fasit_2023 = xr.open_dataset("/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2023/norkyst800-20230101.nc").isel(s_rho = -1, time = 0)
ds_zarr_2023 = open_dataset("/lustre/storeB/project/fou/hi/foccus/datasets/norkystv3_hindcast_2023_surface.zarr", select = "zeta", start = '2013-01-01T00:00:00', end = '2013-01-01T00:00:00')

In [134]:
#flattening the norkyst model
ds_fasit_2023_1d = ds_fasit_2023["zeta"].values.flatten()
ds_fasit_2023_1d.shape

(3153556,)

In [132]:
ds_zarr_2023.shape

(1, 1, 1, 3153556)

In [137]:
diff_2023 = ds_fasit_2023_1d - ds_zarr_2023
replace_nan_2023 = np.where(np.isnan(diff_2023), 0, diff_2023)
if np.any(replace_nan != 0):
    print("not the same zeta values between the norkyst and zarr model")
else:
    print("the values are the same for the zarr and the norkyst model")


the values are the same for the zarr and the norkyst model


In [117]:
#DISCOVER ANEMOI DATASETS LEARNING:


In [110]:
#The dimension of the dataset
ds_zarr.field_shape

(1148, 2747)

In [38]:
#finding the belonging index of a variable
index = ds_zarr.name_to_index["zeta"]
print(index)

0


In [47]:
#checking the statistics for a selected variable
stats = ds_zarr.statistics
stats["mean"]
stats["maximum"]
stats["minimum"]
stats["stdev"]

print(stats["mean"][index], stats["stdev"][index])

-0.12241766349692108 0.4648596113508162


repeat for other variables as well